In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

URL = "https://housing.ucdavis.edu/dining/menus/dining-commons/tercero/"
response = requests.get(URL)

soup = BeautifulSoup(response.content, 'html.parser')

In [2]:
soup.prettify

<bound method Tag.prettify of <!--Missing First, Second and Third Directory-->
<!DOCTYPE html>

<html lang="en" ng-app="weather" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<!--Template:bootstrap-sidemenu-->
<head>
<!--~~~~~~~~~~~~~~~~~~~~~~~~~-->
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-10917221-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-10917221-1');
</script>
<!--~~~~~~~~~~~~~~~~~~~~~~~~~-->
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="UC Davis Student Housing" name="Author">
<meta content="#DAAA00" name="theme-color">
<meta content="#DAAA00" name="msapplication-navbutton-color">
<meta content="yes" name="mobile-web-app-capable"/>
<!--~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
# for i in tab[6].find_all('div', recursive = False)[3].find_all(["img"]):
#     print(i.get('alt'))

In [3]:
tabs = soup.find("div", {"id": "tabs"})

In [4]:
tab = []
for i in tabs.find_all("div", recursive = False): #breakfast, lunch, dinner
    tab.append(i)

In [5]:
tab[0]

<div class="menu_maincontainer three_columns" id="tab1content">
<h3>Sunday, January 16, 2022</h3>
<div class="filter-menu-options"></div>
<div>
<h4>Breakfast</h4>
<h5>Pacific Fusion </h5>
<ul>
<li class="trigger filterCalories400"><span>Chicken Sausage Patties</span>
<ul class="nutrition">
<li>
<div>
</div>
<div>
<h6>Serving Size</h6>
<p>: 2 patties (3 oz)</p>
<h6>Calories</h6>
<p>: 314.5</p>
<h6>Fat (g)</h6>
<p>: 28.9</p>
<h6>Carbohydrates (g)</h6>
<p>: 1.7</p>
<h6>Protein (g)</h6>
<p>: 11.9</p>
<h6>Allergens</h6>
<h6>Ingredients</h6>
<p>Pork Chorizo Sausage (pork, seasoning [spices, salt, paprika, sugar, dried garlic, natural flavor, paprika extract], water)</p>
</div>
</li>
</ul>
</li>
<li class="trigger filterVegetarianfilterHalalfilterNourish filterCalories200"><span>Scrambled Eggs</span>
<ul class="nutrition">
<li>
<div>
<p>Softly scrambled cage-free eggs</p>
</div>
<div>
<img alt="Vegetarian" src="/_images/dining/nutrition-icons/vegetarian.svg" style="max-width:200px;">
<img alt

In [7]:
def get_fooditem(tb):
    menu = []
    newItem = {
            "Name": "",
            "Location": "",
            "Mealtime": "",
            "Date": "",
            "Filters": "",
            "Description": "",
            "Serving_Size": "",
            "Calories": "",
            "Fat": "",
            "Carbohydrates": "",
            "Protein": "",
            "Allergens": "",
            "Ingredients": ""}
    
    curLocation = "" #some dont have any location
    curFilters = [] #some dont have any filters 
    
    for line in tb.find_all():
        if line.name == 'h3':
            curDate = line.text
        if line.name == 'h4':
            curMealTime = line.text
        if line.name == 'h5':
            curLocation = line.text
        if line.name == 'img':
            curFilters.append(line.get('alt'))
        if line.name == "span":
            if newItem["Name"] != '':  
                menu.append(newItem) #adds a new item everytime you hit a span and the name is not empty 
            
            curFilters = [] #some dont have any filters 
            
            newItem = {
                "Name": "",
                "Location": "",
                "Mealtime": "",
                "Date": "",
                "Filters": "",
                "Description": "",
                "Serving_Size": "",
                "Calories": "",
                "Fat": "",
                "Carbohydrates": "",
                "Protein": "",
                "Allergens": "",
                "Ingredients": ""
            }
            
            newItem["Name"]=line.text
            newItem["Filters"] = curFilters
            newItem["Date"]=curDate
            newItem["Mealtime"]=curMealTime
            newItem["Location"]=curLocation
            state = "description"
        if line.name == 'p':
            if state == "description":
                newItem["Description"]=line.text
            if state == "date":
                newItem["Date"] = line.text
            if state == "Serving Size":
                newItem["Serving_Size"]=line.text[2:]
            if state == "Calories":
                newItem["Calories"]=line.text[2:]
            if state == "Fat (g)":
                newItem["Fat"]=line.text[2:]
            if state == "Carbohydrates (g)":
                newItem["Carbohydrates"] = line.text[2:]
            if state == "Protein (g)":
                newItem["Protein"] = line.text[2:]
            if state == "Allergens":
                newItem["Allergens"] = line.text
            if state == "Ingredients":
                newItem["Ingredients"] = line.text
            state = "nothing"
        if line.name == 'h6':
            state = line.text
    
    menu.append(newItem)
    return(menu)

In [8]:
items = [] #index corresponds to days of the week
for i in tab:
    items.append(get_fooditem(i))

In [9]:
len(items)

7

In [442]:
items[0]

[{'Name': 'Beef Street Tacos',
  'Location': 'Pacific Fusion ',
  'Mealtime': 'Dinner',
  'Date': 'Sunday, January 09, 2022',
  'Filters': [],
  'Description': 'Carne Asada beef marinated in Latin spices and wrapped in corn tortillas with fresh cilantro',
  'Serving_Size': '1 SV',
  'Calories': '238.11',
  'Fat': '10.56',
  'Carbohydrates': '12.72',
  'Protein': '21.37',
  'Allergens': 'All major allergens',
  'Ingredients': 'Carne Asada beef (beef, house marinade [orange juice, lime juice, canola oil, garlic, Chipotle Adobo pepper sauce {water, tomato paste, dried Chipotle peppers, distilled vinegar, corn oil. Contains 2% or less of iodized salt, corn starch, onion powder, garlic powder, spices}, dark chili powder, fresh cilantro, paprika, garlic powder, ground cumin, ground coriander, kosher salt, ground black pepper]), corn tortillas (water, stone ground corn masa flour, stone ground fresh corn. Contains less than 2% of cellulose gum, fumaric acid, potassium sorbate, calcium propion

In [375]:
len(items) #for seven days of the week 

7

In [365]:
#first index is day tab
#second index is the divs of each tab so MEALtime breakfast lunch dinner , 0 index is another div so index+1
tab[0].find_all('div', recursive = False)[1].find_all(["span","h6","h5","p","img"])

[<h5>Pacific Fusion </h5>,
 <span>Beef Street Tacos</span>,
 <p>Carne Asada beef marinated in Latin spices and wrapped in corn tortillas with fresh cilantro</p>,
 <h6>Serving Size</h6>,
 <p>: 1 SV</p>,
 <h6>Calories</h6>,
 <p>: 238.11</p>,
 <h6>Fat (g)</h6>,
 <p>: 10.56</p>,
 <h6>Carbohydrates (g)</h6>,
 <p>: 12.72</p>,
 <h6>Protein (g)</h6>,
 <p>: 21.37</p>,
 <h6>Allergens</h6>,
 <p>All major allergens</p>,
 <h6>Ingredients</h6>,
 <p>Carne Asada beef (beef, house marinade [orange juice, lime juice, canola oil, garlic, Chipotle Adobo pepper sauce {water, tomato paste, dried Chipotle peppers, distilled vinegar, corn oil. Contains 2% or less of iodized salt, corn starch, onion powder, garlic powder, spices}, dark chili powder, fresh cilantro, paprika, garlic powder, ground cumin, ground coriander, kosher salt, ground black pepper]), corn tortillas (water, stone ground corn masa flour, stone ground fresh corn. Contains less than 2% of cellulose gum, fumaric acid, potassium sorbate, calciu

In [353]:
sum(lens)

65

In [346]:
fooditems = []
indl = 0
for i in range(len(arr)):
    if arr[i].name == 'span':
        fooditems.append(arr[i])


In [352]:
len(fooditems)

21

In [367]:
for i in tab[6].find_all('div', recursive = False)[3].find_all(["img"]):
    print(i.get('alt'))

Halal
Vegan
Halal
Nourish
Vegetarian
Halal
Halal
Vegetarian
Halal
Vegan
Halal
Halal
Vegan
Halal
Vegan
Halal
Vegan
Halal
Nourish
Vegetarian
Halal
Vegetarian
Halal
Halal
Vegan
Halal
Nourish
Vegan
Halal
Vegetarian
Halal
Vegetarian
Vegan


In [ ]:
menu["foodItems"].append(newItem) #adds last item


with open('db5.json', 'w') as fp:
    json.dump(menu, fp, indent=4, sort_keys=True)